In [72]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import os
import math

from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
import scipy.stats as stats

#from mpltools import special ##leon

plt.style.use('seaborn-poster')

# DEFINE DATA CLASS
class RawData(object):
    def __init__(self,timestamp,keytime,ttime,keypress,targetid,target_lon,target_lat,target_x,target_y,target_z,hmd_rx,hmd_ry,hmd_rz):
        self.timestamp=timestamp # 1 (default time?)
        self.keytime=keytime # 2 +Time.deltaTime
        self.ttime=ttime # 3 Time.time
        self.keypress=keypress # 4
        self.targetid=targetid # 5
        self.target_lon=target_lon
        self.target_lat=target_lat
        self.target_x=target_x
        self.target_y=target_y
        self.target_z=target_z
        self.hmd_rx=hmd_rx # 11
        self.hmd_ry=hmd_ry # 12
        self.hmd_rz=hmd_rz # 13

class TimeData(object):
    def __init__(self,targetid,target_lon,target_lat,deltatime):
        self.targetid=targetid
        self.target_lon=target_lon
        self.target_lat=target_lat
        self.deltatime=deltatime


## READ DATA
def read_data(file):
    raw_data=[]
    print("Reading %s"%file)
    seperator="\t"
    with open(file,"r") as fp:
        for line in fp:
            if line.startswith("Time"):
                continue     ## Skip the Header
            l=line[:-1]   ## Delete the Last Element then Return
            f=l.split(seperator)   ## Divide the Raw Data to Pieces
            if(int(f[3])==1):   ## 4.When Finish Targeting and Press Key
                raw_data.append(RawData(f[0],float(f[1]),float(f[2]),float(f[3]),float(f[4]),float(f[5]),float(f[6]),float(f[7]),float(f[8]),float(f[9]),float(f[10]),float(f[11]),float(f[12])))
            else: 
                raw_data.append(RawData(f[0],float(f[1]),float(f[2]),float(f[3]),float(f[4]),0,0,0,0,0,float(f[10]),float(f[11]),float(f[12])))
                
    return raw_data



# Input Raw Data, Each Tester, return aim_time[[[,,,],,,,,][,,,,,][,,,,,]] 3-layer array
# Every Target Aiming Time in aim_time[lat][lon] = [first, second, third]
def aim_time_filter(input_raw_data):
    aim_time={}
    filtered_data=[a for a in input_raw_data if a.keypress==1]

    for i in range(len(filtered_data)-1):
        if(filtered_data[i+1].targetid!=filtered_data[i].targetid):
            continue # when resuming back
        if((i <= len(filtered_data)-3) and (filtered_data[i-1].targetid==filtered_data[i+2].targetid)):
            continue # when two same targets appear one after one
        delta_time=filtered_data[i+1].ttime-filtered_data[i].ttime
        #print(delta_time, filtered_data[i].targetid)

        key1=str(int(filtered_data[i].target_lat))
        key2=str(int(filtered_data[i].target_lon))
        if(key1 in aim_time)==False:
            aim_time[key1]={}
        if(key2 in aim_time[key1])==False:
            aim_time[key1][key2]=[]
        
        aim_time[key1][key2].append(TimeData(filtered_data[i].targetid,filtered_data[i].target_lon,filtered_data[i].target_lat,delta_time))
    
    aim_time[str(0)][str(0)]=[]   # Special (0,0) Not Exist but for data processing, add a pseudo datum
    aim_time[str(0)][str(0)].append(TimeData(float(21),float(0),float(0),float(0)))

    return aim_time   # aim_time is 3*3*7 array{}

def extract_time(input_time):
    #lat = [a for a in input_time]
    #lon = [b for b in input_time[lat]]
    #target = [c.deltatime for c in input_time[lon] ]
    
    time_dict = {}
    #time_flat = [] # not useful, not sorted
    
    lat_order = ['-30','0','30']
    lon_order = ['0','30','60','90','120','150','180']
    
    for lat in lat_order:
        time_dict[lat] ={}
        #time_lon = []
        for lon in lon_order:
            time_dict[lat][lon] = [target.deltatime for target in input_time[lat][lon]]
            #time_target = [target.deltatime for target in input_time[lat][lon]]
            #time_lon.append(time_target)
        #time_flat.append(time_lon)

    return time_dict#, time_flat

def flatten_data(input_data):
    
    flat_data = []
    
    for lat in input_data:
        data_lon = []
        for lon in input_data[lat]:
            data_target = [target for target in input_data[lat][lon]]
            data_lon.append(data_target)
        flat_data.append(data_lon)
    
    return flat_data


def sort_data(input_data):
    
    sorted_data = []
    
    lat_order = ['-30','0','30']
    lon_order = ['0','30','60','90','120','150','180']
    
    for lat in lat_order:
        data_lon = []
        for lon in lon_order:
            data_target = [target for target in input_data[lat][lon]]
            data_lon.append(data_target)
        sorted_data.append(data_lon)
        
    return sorted_data
    
    
def extract_time_speed(input_time):
    time_dict = {}
    speed_dict = {}
    time_flat = []
    speed_flat = []

    key1=[-30, 0, 30]
    key2=[0,30,60,90,120,150,180]
    k1=[str(a) for a in key1]
    k2=[str(a) for a in key2]



    for i in range(len(input_time)):
        time_dict[k1[i]] = {}
        speed_dict[k1[i]] = {}
        time_flat[k1[i]] = []
        speed_flat[k1[i]] = []

        for j in range(len(input_time[i])):
            time_dict[k1[i]][k2[i]] = {}
            speed_dict[k1[i]][k2[i]] = {}
            time_flat[k1[i]][k2[i]] = []
            speed_flat[k1[i]][k2[i]] = []

            for k in range(len(input_time[str(key1[i])][str(key2[j])])):
                if(input_time[str(key1[i])][str(key2[j])][k]==0):
                    #r.append(0) # speed = 0
                    speed[str(key1)][str(key2)].append(0)
                    continue

                angle = math.sqrt( (30*i-30)*(30*i-30) + (30*j)*(30*j) ) # for 1,2 approximately
                speed[str(key1)][str(key2)].append( angle / input_time[str(key1[i])][str(key2[j])][k] )

    return speed   
    



# Read the Time
# Calculate the Mean and STD
def time_stat(path,raw,fig): # return
    raw_data=[]
    raw_data=read_data(path,raw_data)
    aim_time=aim_time_filter(raw_data)
    

    key1=['-30','0','30']    # 3 lat
    if (len(aim_time) == 1):
        key1=['0']    # 1 lat
    key2=[0,30,60,90,120,150,180]
    

    target_mean_s=[]
    target_std_s=[]
    for k1 in range(len(key1)):
        m1=[]
        s1=[]
        for k2 in range(len(key2)):
            if(k1==1 and k2==0):
            #if(k2==0):
                m1.append(0)
                s1.append(0)
                continue
            data=[a.delta_time for a in aim_time[key1[k1]][str(key2[k2])]]
            
            m1.append(np.mean(data))
            s1.append(np.std(data))
   
        target_mean_s.append(m1)
        target_std_s.append(s1)

    return target_mean_s, target_std_s    # 3*7 array[]


# Calculate the Mean and STD
def data_stat(input_data): # return

    key1=['-30','0','30']    # 3 lat
    if (len(input_data) == 1):
        key1=['0']    # 1 lat
    key2=[0,30,60,90,120,150,180]
    

    target_mean_s=[]
    target_std_s=[]
    for k1 in range(len(key1)):
        m1=[]
        s1=[]
        for k2 in range(len(key2)):
            if(k1==1 and k2==0):
            #if(k2==0):
                m1.append(0)
                s1.append(0)
                continue
            data=[a for a in input_data[key1[k1]][str(key2[k2])]]
            
            m1.append(np.mean(data))
            s1.append(np.std(data))
   
        target_mean_s.append(m1)
        target_std_s.append(s1)

    return target_mean_s, target_std_s    # 3*7 array[]

def mean_stat(input_time): # return

    key1=['-30','0','30']    # 3 lat
    if (len(input_time) == 1):
        key1=['0']    # 1 lat
    key2=[0,30,60,90,120,150,180]
    

    target_mean_s=[]
    target_std_s=[]
    for k1 in range(len(key1)):
        m1=[]
        s1=[]
        for k2 in range(len(key2)):
            if(k1==1 and k2==0):
            #if(k2==0):
                m1.append(0)
                s1.append(0)
                continue
            data=[a.deltatime for a in input_time[key1[k1]][str(key2[k2])]]
            
            m1.append(np.mean(data))
            s1.append(np.std(data))
   
        target_mean_s.append(m1)
        target_std_s.append(s1)

    return target_mean_s, target_std_s    # 3*7 array[]

def time_to_speed(input_time):
    
    speed={}
    
    for lat in input_time:
        speed[lat]={}
        for lon in input_time[lat]:
            speed[lat][lon]=[]
            for target in range(len(input_time[lat][lon])):
                if (input_time[lat][lon][target]==0):
                    #r.append(0) # speed = 0
                    speed[lat][lon].append(0)
                    continue
                
                angle = math.sqrt( int(lat)**2 + int(lon)**2 )
                speed[lat][lon].append( angle / input_time[lat][lon][target] )

    return speed    # 3*3*7 array{}

def time_to_speed_ac(input_time):
    
    key1=[-30, 0, 30]
    key2=[0,30,60,90,120,150,180]
    
    speed={}
    
    for i in range(len(input_time)):
        speed[str(key1[i])]={}
        for j in range(len(input_time[str(key1[i])])):
            speed[str(key1[i])][str(key2[j])]=[]
            for k in range(len(input_time[str(key1[i])][str(key2[j])])):
                if(input_time[str(key1[i])][str(key2[j])][k]==0):
                    #r.append(0) # speed = 0
                    speed[str(key1)][str(key2)].append(0)
                    continue
                
                angle = math.degrees(math.acos(math.cos(math.radians(key1[i])) * math.cos(math.radians(key2[j]))))
                speed[str(key1[i])][str(key2[j])].append( angle / input_time[str(key1[i])][str(key2[j])][k] )

    return speed    # 3*3*7 array{}


def time_to_speed_ap(input_time):
    
    key1=[-30, 0, 30]
    key2=[0,30,60,90,120,150,180]
    
    speed={}
    
    for i in range(len(input_time)):
        speed[str(key1[i])]={}
        for j in range(len(input_time[str(key1[i])])):
            speed[str(key1[i])][str(key2[j])]=[]
            for k in range(len(input_time[str(key1[i])][str(key2[j])])):
                if(input_time[str(key1[i])][str(key2[j])][k]==0):
                    #r.append(0) # speed = 0
                    speed[str(key1[i])][str(key2[j])].append(0)
                    continue

                angle = math.sqrt( (30*i-30)*(30*i-30) + (30*j)*(30*j) ) # for 1,2 approximately
                speed[str(key1[i])][str(key2[j])].append( angle / input_time[str(key1[i])][str(key2[j])][k] )

    return speed    # 3*3*7 array{}





In [73]:

###################### choose user
users=[1,2,3,4,5,9,10,14,6,12,13,8,11,7]#after 13
#users=[14,16,17,18,19,15]
######################


for uid in range(len(users)):
    baseFolder=str(users[uid])
    #idx=[3,4,5]
    idx=[1,2]

baseFolder = "1"
raw_data = read_data("../Users/"+baseFolder+"/"+str(idx[0])+".txt")
aim_time = aim_time_filter(raw_data)
time_dict = extract_time(aim_time)

time_flat = flatten_data(time_dict)

speed_dict = time_to_speed_ap(time_dict)
speed_test = time_to_speed(time_dict)
sorted_speed = flatten_data(speed_dict)

if(True):
    for x in speed_dict:
        print ('--', x, '--')
        for y in speed_dict[x]:
            print (y,':', speed_dict[x][y])
    
    for x in speed_test:
        print ('--', x, '--')
        for y in speed_test[x]:
            print (y,':', speed_test[x][y])

    for x in range(3):
        print ('--', x, '--')
        for y in range(7):
            print (y,':')
            for z in range(len(sorted_speed[x][y])):
                print (z,',', sorted_speed[x][y][z])

print(np.array(sorted_speed).shape)


Reading ../Users/1/1.txt
-- -30 --
0 : [30.318342597270707, 21.679433444139203, 25.471217524197858, 26.995410780169443]
30 : [35.04287343784071, 21.933726346064567, 30.821944693929492, 33.91399430151371]
60 : [36.14528763672233, 33.29959758004157, 35.895783029212815, 41.29650290876438]
90 : [52.002592668449104, 51.35512900181527, 56.135106393521596, 52.194283563519434]
120 : [73.0140893504105, 38.60826792200817, 55.274452037058815, 48.57570246957648]
150 : [58.28339000525197, 34.6385094442696, 64.32741186197832, 59.12820741671329]
180 : [51.17585840735469, 55.05426775748128, 54.299067429091004, 57.14375772184717]
-- 0 --
0 : [0]
30 : [33.20053120849996, 38.422131147541016, 36.403349108116004, 28.506271379702607]
60 : [46.52244708071504, 15.058351110553513, 54.431642928424516, 31.801558276355223]
90 : [62.27080882861799, 62.2794270292727, 70.49976500078479]
120 : [79.5070562512425, 72.3283708034491, 59.958029379434706, 62.12144742972582]
150 : [52.87088928835744, 60.207112466886116, 66.